In [1]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
# import tensorflow as tf

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Data Preprocessing

## Load the housing data using Pandas

In [2]:
 # Import our input dataset
df = pd.read_csv('kc_house_data.csv')
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Clean the Data

In [3]:
# check data types
df.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [4]:
# convert date column to datetime
df['date']= pd.to_datetime(df['date'])

# check data types
df.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

In [5]:
# add separate columns for the sale year and month
df['yr_sold'] = df['date'].dt.year
df['month_sold'] = df['date'].dt.month

# drop the original date column
df.drop('date', axis=1, inplace=True)

# convert price to price in thousands
df['price(000s)'] = df['price']/1000

# drop the original price column
df.drop('price', axis=1, inplace=True)

# show updated data frame
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold,month_sold,price(000s)
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,1955,0,98178,47.5112,-122.257,1340,5650,2014,10,221.9
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,1951,1991,98125,47.7210,-122.319,1690,7639,2014,12,538.0
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,1933,0,98028,47.7379,-122.233,2720,8062,2015,2,180.0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,1965,0,98136,47.5208,-122.393,1360,5000,2014,12,604.0
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,1987,0,98074,47.6168,-122.045,1800,7503,2015,2,510.0


In [6]:
 # Encoding the yr_renovated column using a custom function
def encode_renovated(yr_renovated):
    """
    This function encodes renovation status by setting any year as 1 and 0 as 0.
    """
    if yr_renovated > 0:
        return 1
    else:
        return 0

# Call the encode_renovated function on the renovated column
df["renovated"] = df["yr_renovated"].apply(encode_renovated)


# Encoding the sqft_basement column using a custom function
def encode_basement(sqft_basement):
    """
    This function encodes basement status by setting any square footage as 1 and 0 as 0.
    """
    if sqft_basement > 0:
        return 1
    else:
        return 0

# Call the encode_basement function on the basement column
df["basement"] = df["sqft_basement"].apply(encode_basement)

# Review the DataFrame 
df.head()


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold,month_sold,price(000s),renovated,basement
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,98178,47.5112,-122.257,1340,5650,2014,10,221.9,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,98125,47.7210,-122.319,1690,7639,2014,12,538.0,1,1
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,98028,47.7379,-122.233,2720,8062,2015,2,180.0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,98136,47.5208,-122.393,1360,5000,2014,12,604.0,0,1
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,98074,47.6168,-122.045,1800,7503,2015,2,510.0,0,0


In [7]:
#count rows before dropping rows with duplicates or NA values
print(len(df))

# drop duplicate rows
df.drop_duplicates(subset="id", keep='first', inplace=True)

# drop rows containing NA values
df.dropna(axis=0, inplace=True)

#count rows after dropping rows with duplicates or NA values
print(len(df))

# show updated data frame
df.head()

21613
21436


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold,month_sold,price(000s),renovated,basement
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,98178,47.5112,-122.257,1340,5650,2014,10,221.9,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,98125,47.7210,-122.319,1690,7639,2014,12,538.0,1,1
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,98028,47.7379,-122.233,2720,8062,2015,2,180.0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,98136,47.5208,-122.393,1360,5000,2014,12,604.0,0,1
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,98074,47.6168,-122.045,1800,7503,2015,2,510.0,0,0


### Identify and Remove Outliers

In [8]:

#identify outliers in price data using boxplots
#boxplot = df.boxplot(column="price(000s)", showcaps=True)

In [9]:
# Determine which data points are outlier (outside of the 1.5*IQR range)
prices = df['price(000s)']
quartiles = np.quantile(prices,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)

# identify outliers
# potential_outliers = [price for price in prices if price < lower_bound or price > upper_bound]

# calculate percent of data to be dropped
# print(len(potential_outliers)/len(df)*100)

# drop rows with price value lower than lower_bound (outliers)
df = df[df['price(000s)'] >= lower_bound]

#length of updated data frame
print(len(df))

# drop rows with price value higher than upper_bound (outliers)
df = df[df['price(000s)'] <= upper_bound]

# length of updated data frame
print(len(df))

# show updated df
df.head()

21436
20297


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold,month_sold,price(000s),renovated,basement
0,7129300520,3,1.00,1180,5650,1.0,0,0,3,7,...,98178,47.5112,-122.257,1340,5650,2014,10,221.9,0,0
1,6414100192,3,2.25,2570,7242,2.0,0,0,3,7,...,98125,47.7210,-122.319,1690,7639,2014,12,538.0,1,1
2,5631500400,2,1.00,770,10000,1.0,0,0,3,6,...,98028,47.7379,-122.233,2720,8062,2015,2,180.0,0,0
3,2487200875,4,3.00,1960,5000,1.0,0,0,5,7,...,98136,47.5208,-122.393,1360,5000,2014,12,604.0,0,1
4,1954400510,3,2.00,1680,8080,1.0,0,0,3,8,...,98074,47.6168,-122.045,1800,7503,2015,2,510.0,0,0


In [15]:
# Determine which data points are outlier (outside of the 1.5*IQR range)
bedrooms = df['bedrooms']
quartiles = np.quantile(prices,[.25,.75])
iqr = quartiles[1]-quartiles[0]
lower_bound = quartiles[0]-(1.5*iqr)
upper_bound = quartiles[1]+(1.5*iqr)

# identify outliers
# potential_outliers = [price for price in prices if price < lower_bound or price > upper_bound]

# calculate percent of data to be dropped
# print(len(potential_outliers)/len(df)*100)

# drop rows with number of bedrooms lower than lower_bound (outliers)
df = df[df['bedrooms'] >= lower_bound]

#length of updated data frame
print(len(df))

# drop rows with number of bedrooms higher than upper_bound (outliers)
df = df[df['bedrooms'] <= upper_bound]

# length of updated data frame
print(len(df))

# show updated df
df.head()


33

In [15]:
# # plot price of all houses
# x_values = df['id']
# y_values = df['price(000s)']
# plt.scatter(x_values, y_values)


# # identify outliers in subsets using boxplots
# df.boxplot(by='bedrooms', column=["price(000s)"], grid=False)
# df.boxplot(by='bathrooms', column=["price(000s)"], grid=False)
# df.boxplot(by='floors', column=["price(000s)"], grid=False)
# df.boxplot(by='waterfront', column=["price(000s)"], grid=False)
# df.boxplot(by='view', column=["price(000s)"], grid=False)
# df.boxplot(by='condition', column=["price(000s)"], grid=False)
# df.boxplot(by='grade', column=["price(000s)"], grid=False)
# df.boxplot(by='yr_built', column=["price(000s)"], grid=False)
# df.boxplot(by='zipcode', column=["price(000s)"], grid=False)
# df.boxplot(by='yr_sold', column=["price(000s)"], grid=False)

In [11]:
 # Scaling the numeric columns
data_scaled = StandardScaler().fit_transform(df[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
       'floors', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated',
       #'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'yr_sold', 'month_sold']])

# Review the scaled data
data_scaled

array([[-0.3621819 , -1.48174323, -1.02935311, ..., -0.25616169,
        -0.68366252,  1.09469788],
       [-0.3621819 ,  0.27540177,  0.76310217, ..., -0.1814131 ,
        -0.68366252,  1.73776002],
       [-1.45888577, -1.48174323, -1.55806294, ..., -0.16551634,
         1.46270998, -1.47755066],
       ...,
       [-1.45888577, -1.83317223, -1.2356789 , ..., -0.39306925,
        -0.68366252, -0.19142639],
       [-0.3621819 ,  0.62683077, -0.48774792, ..., -0.42012757,
         1.46270998, -1.79908173],
       [-1.45888577, -1.83317223, -1.2356789 , ..., -0.4174969 ,
        -0.68366252,  1.09469788]])

In [12]:
# Create a DataFrame of the scaled data
data_scaled_df = pd.DataFrame(data_scaled, columns=['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 
                                                'floors', 'view', 'condition', 'grade', 'sqft_above',
                                                'sqft_basement', 'yr_built', 'yr_renovated',
                                                #'zipcode', 'lat', 'long',
                                                'sqft_living15', 'sqft_lot15', 'yr_sold', 'month_sold'])

# Replace the original data with the columns of information from the scaled Data
df["bedrooms"] = data_scaled_df["bedrooms"]
df["bathrooms"] = data_scaled_df["bathrooms"]
df["sqft_living"] = data_scaled_df["sqft_living"]
df["sqft_lot"] = data_scaled_df["sqft_lot"]
df["floors"] = data_scaled_df["floors"]
df["view"] = data_scaled_df["view"]
df["condition"] = data_scaled_df["condition"]
df["grade"] = data_scaled_df["grade"]
df["sqft_above"] = data_scaled_df["sqft_above"]
df["sqft_basement"] = data_scaled_df["sqft_basement"]
df["yr_built"] = data_scaled_df["yr_built"]
df["yr_renovated"] = data_scaled_df["yr_renovated"]
#df["zipcode"] = data_scaled_df["zipcode"]
#df["lat"] = data_scaled_df["lat"]
#df["long"] = data_scaled_df["long"]
df["sqft_living15"] = data_scaled_df["sqft_living15"]
df["sqft_lot15"] = data_scaled_df["sqft_lot15"]
df["yr_sold"] = data_scaled_df["yr_sold"]
df["month_sold"] = data_scaled_df["month_sold"]


# Review the DataFrame
df.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,zipcode,lat,long,sqft_living15,sqft_lot15,yr_sold,month_sold,price(000s),renovated,basement
0,7129300520,-0.362182,-1.481743,-1.029353,-0.223532,-0.889079,0,-0.27027,-0.630048,-0.515778,...,98178,47.5112,-122.257,-0.948985,-0.256162,-0.683663,1.094698,221.9,0,0
1,6414100192,-0.362182,0.275402,0.763102,-0.183976,0.971487,0,-0.27027,-0.630048,-0.515778,...,98125,47.7210,-122.319,-0.380357,-0.181413,-0.683663,1.737760,538.0,1,1
2,5631500400,-1.458886,-1.481743,-1.558063,-0.115448,-0.889079,0,-0.27027,-0.630048,-1.478954,...,98028,47.7379,-122.233,1.293034,-0.165516,1.462710,-1.477551,180.0,0,0
3,2487200875,0.734522,1.329689,-0.023515,-0.239683,-0.889079,0,-0.27027,2.462221,-0.515778,...,98136,47.5208,-122.393,-0.916492,-0.280589,-0.683663,1.737760,604.0,0,1
4,1954400510,-0.362182,-0.076027,-0.384585,-0.163154,-0.889079,0,-0.27027,-0.630048,0.447397,...,98074,47.6168,-122.045,-0.201645,-0.186524,1.462710,-1.477551,510.0,0,0


# Data Splitting

In [16]:
 # Remove housing price target from features data
y = df["price(000s)"].values
X = df.drop(columns="price(000s)").values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=15, stratify=y)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.